<a href="https://colab.research.google.com/github/dongjaeseo/NLP_study/blob/main/courtsbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchsummary

In [ ]:
import numpy as np
from torchsummary import summary
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

In [ ]:
print(transformers.__version__)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, max_length=32):
        self.texts = texts
        self.labels = labels
        self.max_length = max_length
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = int(self.labels[idx])

        # Tokenize the text
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Your dataset
# texts = ['i am happy', 'i am sad', 'movie is good', 'hes a bad guy', 'what a good day', 'gloomy day', 'I hate this movie', 'i just love this coffee', 'i dont want to go to class today', 'creepy idea', 'lovely cat', 'great idea']
# labels = [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1]
texts = ['what slots are available for this court on 26th?',
         'can I know which timings are available on Friday?',
         'Which are the possible slots for next Wednesday?',
         'when can i book court on 17th?',
         'how can i contact with you?',
         'i would like to call someone incharge',
         'I dont understand the process. Could I contact you?',
         'Can you send me an email?',
         'how can i make payment?',
         'can I pay through bank account transfer?',
         'I want to make monthly subscription',
         'Which payment options do you offer?']

labels = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]


# Create an instance of your CustomDataset
custom_dataset = CustomDataset(texts, labels)

# Create a DataLoader
batch_size = 2  # You can adjust this based on your needs
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Example usage
for batch in custom_dataloader:
    print(batch)

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm

# Sample dataset class
class CustomDataset(Dataset):
    def __init__(self, texts, labels, max_length=32):
        self.texts = texts
        self.labels = labels
        self.max_length = max_length
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = int(self.labels[idx])

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Your dataset
texts = ['What slots are available for booking this Friday?',
         'can I know which timings are available on Friday?',
         'Which are the possible slots for next Wednesday?',
         'when can i book court on 17th?',
         'how can i contact with you?',
         'i would like to call someone in charge',
         'I dont understand the process. Could I contact you?',
         'Can you send me an email?',
         'how can i make payment?',
         'can I pay through bank account transfer?',
         'I want to make a monthly subscription',
         'Which payment options do you offer?']

labels = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]

custom_dataset = CustomDataset(texts, labels)
batch_size = 2
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Load pre-trained BERT model for sequence classification
model_name = 'bert-base-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming 3 classes

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 20  # Adjust as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(custom_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        batch_labels = batch['labels'].to(torch.long).to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask=attention_mask).logits
        loss = criterion(logits, batch_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(custom_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}")

# Save the fine-tuned model
model.save_pretrained('fine_tuned_bert_model')

In [ ]:
# Load the fine-tuned BERT model
fine_tuned_model = BertForSequenceClassification.from_pretrained('fine_tuned_bert_model', num_labels=3)  # Assuming 3 classes

# Set the model to evaluation mode
fine_tuned_model.eval()

# Example texts for inference
example_texts = [
    'what slots are available for this court on 26th?',
    'can I know which timings are available on Friday?',
    'what are possible timings on next Sunday?',
    'Can you send me an email?',
    'I would like to talk to you',
    'Can we have a talk',
    'Please drop me an email',
    'Please drop me a message',
    'i want to message you',
    'how to make transaction',
    'payment portal isnt there',
    'I want to make payment',

    'when can i pay',
    'i want to talk regarding payment',
    'i want to talk regarding when to book'
]

# Tokenize and process the example texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer(example_texts, return_tensors='pt', truncation=True, padding=True, max_length=32)

# Move inputs to the appropriate device (CPU or GPU)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Move the model to the same device as the inputs
fine_tuned_model.to(device)

with torch.no_grad():
    logits = fine_tuned_model(**inputs).logits

# Apply softmax to get probabilities
probs = torch.nn.functional.softmax(logits, dim=-1)

# Get predicted labels
predicted_labels = torch.argmax(probs, dim=1).cpu().numpy()

# Print the results
for i, text in enumerate(example_texts):
    print(f"Text: {text}")
    print(f"Predicted Label: {predicted_labels[i]}")
    print(f"Probabilities: {probs[i]}")
    print()